In [93]:
import torch
import random

# Agent class to represent predators and preys
class Agent:
    def __init__(self, agent_type, x, y):
        self.agent_type = agent_type  # 1 for prey, 2 for predator
        self.x = x  # x position on the grid
        self.y = y  # y position on the grid



# Game environment class to manage agents, grid, and game loop
class GameEnvironment:
    def __init__(self, grid_size, num_predators, num_preys):
        self.grid_size = grid_size  # Size of the grid (e.g. (200, 100))
        self.num_predators = num_predators  # Number of predators
        self.num_preys = num_preys  # Number of preys
        self.grid = torch.zeros(grid_size, dtype=torch.int)  # Initialize the grid with zeros using PyTorch tensors
        self.predators = []  # List to store predator agents
        self.preys = []  # List to store prey agents
        
        self.init_agents()  # Initialize agents on the grid

    # Initialize predators and preys on the grid
    def init_agents(self):
        agent_positions = set()  # Set to store occupied positions on the grid

        # Initialize predators
        for _ in range(self.num_predators):
            x, y = self.get_random_position(agent_positions)  # Get a random unoccupied position on the grid
            agent_positions.add((x, y))  # Add the position to the set of occupied positions
            predator = Agent(agent_type=2, x=x, y=y)  # Create a predator agent
            self.predators.append(predator)  # Add the predator to the list of predators
            self.grid[x, y] = 2  # Update the grid to indicate the predator's position

        # Initialize preys
        for _ in range(self.num_preys):
            x, y = self.get_random_position(agent_positions)  # Get a random unoccupied position on the grid
            agent_positions.add((x, y))  # Add the position to the set of occupied positions
            prey = Agent(agent_type=1, x=x, y=y)  # Create a prey agent
            self.preys.append(prey)  # Add the prey to the list of preys
            self.grid[x, y] = 1  # Update the grid to indicate the prey's position

    # Get a random unoccupied position on the grid
    def get_random_position(self, agent_positions):
        x, y = random.randint(0, self.grid_size[0]-1), random.randint(0, self.grid_size[1]-1)
        # Ensure that the chosen position is not occupied by another agent
        while (x, y) in agent_positions:
            x, y = random.randint(0, self.grid_size[0]-1), random.randint(0, self.grid_size[1]-1)
        return x, y

# Example usage
grid_size = (10, 10)
num_predators = 2
num_preys = 5

game_env = GameEnvironment(grid_size, num_predators, num_preys)
game_env.grid

tensor([[1, 2, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)

In [115]:
class Predator(Agent):
    def __init__(self, x, y):
        super().__init__(agent_type=2, x=x, y=y)
        self.life_points = 80


    def propose_move(self, action, game_env):
        x, y = self.x, self.y
        if action == "top":
            x -= 1
        elif action == "down":
            x += 1
        elif action == "left":
            y -= 1
        elif action == "right":
            y += 1

        x = x % game_env.grid_size[0]
        y = y % game_env.grid_size[1]

        return x, y
    

    def move(self, x, y, game_env):
        game_env.grid[self.x, self.y] = 0
        game_env.grid[x, y] = 2
        self.x, self.y = x, y


    def update_vision(self, game_env):
        vision = torch.zeros((5, 5), dtype=torch.int)
        for i in range(-2, 3):
            for j in range(-2, 3):
                x, y = (self.x + i) % game_env.grid_size[0], (self.y + j) % game_env.grid_size[1]
                vision[i + 2, j + 2] = game_env.grid[x, y]

        vision[self.x, self.y] = 2

        return vision

    
p = Predator(9,9)
p.update_vision(game_env)

IndexError: index 9 is out of bounds for dimension 0 with size 5